# General Use

## Find the difference between two DICOM files

In [7]:
import difflib
import pydicom
import tabulate

from pydicom.data import get_testdata_file
from pydicom.valuerep import PersonNameUnicode
from pydicom import dcmread, dcmwrite
from pydicom.filebase import DicomFileLike

from pprint import pprint
from pathlib import Path
from io import BytesIO

from IPython.display import display, HTML, FileLink

In [2]:
filename_mr = get_testdata_file('MR_small.dcm')
filename_ct = get_testdata_file('CT_small.dcm')

datasets = tuple([pydicom.dcmread(filename, force=True) for filename in (filename_mr, filename_ct)])

# Reference of pydicom.dataset.filedataset
#   https://pydicom.github.io/pydicom/dev/reference/generated/pydicom.dataset.FileDataset.html
for data in datasets:
    print(f'############ {data.filename} ############')
    pprint({k:v for k,v in data.to_json_dict().items() if k.startswith('0008')})

############ /Users/painflat/.local/share/virtualenvs/DICOM--xoABAxb/lib/python3.9/site-packages/pydicom/data/test_files/MR_small.dcm ############
{'00080008': {'Value': ['DERIVED', 'SECONDARY', 'OTHER'], 'vr': 'CS'},
 '00080012': {'Value': ['20040826'], 'vr': 'DA'},
 '00080013': {'Value': ['185434'], 'vr': 'TM'},
 '00080014': {'Value': ['1.3.6.1.4.1.5962.3'], 'vr': 'UI'},
 '00080016': {'Value': ['1.2.840.10008.5.1.4.1.1.4'], 'vr': 'UI'},
 '00080018': {'Value': ['1.3.6.1.4.1.5962.1.1.4.1.1.20040826185059.5457'],
              'vr': 'UI'},
 '00080020': {'Value': ['20040826'], 'vr': 'DA'},
 '00080021': {'vr': 'DA'},
 '00080022': {'vr': 'DA'},
 '00080030': {'Value': ['185059'], 'vr': 'TM'},
 '00080031': {'vr': 'TM'},
 '00080032': {'vr': 'TM'},
 '00080050': {'vr': 'SH'},
 '00080060': {'Value': ['MR'], 'vr': 'CS'},
 '00080070': {'Value': ['TOSHIBA_MEC'], 'vr': 'LO'},
 '00080080': {'Value': ['TOSHIBA'], 'vr': 'LO'},
 '00080090': {'vr': 'PN'},
 '00080201': {'Value': ['-0400'], 'vr': 'SH'},
 '

In [3]:
rep = []
for dataset in datasets:
    lines = str(dataset).split("\n")
    lines = [line + "\n" for line in lines]  # add the newline to end
    rep.append(lines)


html_diff = difflib.HtmlDiff().make_file(rep[0], rep[1])
Path('diff.html').write_text(html_diff)
FileLink('diff.html')

/Users/painflat/Projects/study/DICOM/diff.html

## Display unicode person names

In [4]:
default_encoding = 'iso8859'

person_names = [
    PersonNameUnicode(
        b"Yamada^Tarou=\033$B;3ED\033(B^\033$BB@O:"
        b"\033(B=\033$B$d$^$@\033(B^\033$B$?$m$&\033(B",
        [default_encoding, 'iso2022_jp']),  # DICOM standard 2008-PS3.5 H.3 p98
    PersonNameUnicode(
        b"Wang^XiaoDong=\xcd\xf5\x5e\xd0\xa1\xb6\xab=",
        [default_encoding, 'GB18030']),  # DICOM standard 2008-PS3.5 J.3 p 105
    PersonNameUnicode(
        b"Wang^XiaoDong=\xe7\x8e\x8b\x5e\xe5\xb0\x8f\xe6\x9d\xb1=",
        [default_encoding, 'UTF-8']),  # DICOM standard 2008-PS3.5 J.1 p 104
    PersonNameUnicode(
        b"Hong^Gildong=\033$)C\373\363^\033$)C\321\316\324\327="
        b"\033$)C\310\253^\033$)C\261\346\265\277",
        [default_encoding, 'euc_kr']),  # DICOM standard 2008-PS3.5 I.2 p 101
]


data = []
for name in person_names:
    data.append([name.encodings[1], name.decode(name.encodings[1])])

table = tabulate.tabulate(data, headers=['Encode', 'Name'], tablefmt='html')
display(HTML(table))
    

Encode,Name
iso2022_jp,Yamada^Tarou=山田^太郎=やまだ^たろう
GB18030,Wang^XiaoDong=王^小东
UTF-8,Wang^XiaoDong=王^小東
euc_kr,Hong^Gildong=洪^吉洞=홍^길동


## Show working with memory-based dataset

In [13]:
def write_dataset_to_bytes(dataset):
    with BytesIO() as buffer:
        memory_dataset = DicomFileLike(buffer) # file like object
        dcmwrite(memory_dataset, dataset)
        memory_dataset.seek(0)
        return memory_dataset.read() # return data in bytes


def adapt_dataset_from_bytes(blob):
    dataset = dcmread(BytesIO(blob))
    dataset.is_little_endian = False
    dataset.PatientName = 'Bond^James'
    dataset.PatientID = '007'
    return dataset


class DummyDataBase:
    def __init__(self):
        self._blobs = {}

    def save(self, name, blob):
        self._blobs[name] = blob

    def load(self, name):
        return self._blobs.get(name)

    
file_path = get_testdata_file('CT_small.dcm')
db = DummyDataBase()

dataset = dcmread(file_path)

ds_bytes = write_dataset_to_bytes(dataset)
print(type(ds_bytes))
print(ds_bytes[:100])
db.save('dataset', ds_bytes)

read_bytes = db.load('dataset')
read_dataset = adapt_dataset_from_bytes(read_bytes)
print(type(read_dataset))
print(str(read_dataset)[:100])


<class 'bytes'>
b'II*\x00T\x18\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
<class 'pydicom.dataset.FileDataset'>
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  U


# Image Processing

# Input & Output

# Metadata Processing

# DICOM Network Protocol

# Introduction of Orthanc